In [1]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [3]:

load_dotenv()
KEY = os.getenv("OPENAI_API_KEY") 
llm = ChatOpenAI(openai_api_key= KEY, model_name= "gpt-3.5-turbo", temperature ="0.5")

e:\GPT BootCamp\OPENAI\MCQGenerator\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [15]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

# Physiology

In [15]:
THEME = "Respiratory"
TOPICS = ["cells that carry out gas exchange", "Surfactants", "Asthama", "Pneumothorax", "how birds breathe", "how amphibians breathe", "how insects breathe", "Halden-shift", "Bohr-Shift", "Bronchiolies",
          "Structure of the mamalian lung", "General Respiratory function", "Key component for respiration"]

In [27]:
THEME = "Circulation"
TOPICS = ["Open Circulatory Sytem", "Closed Circulatory Sytem", "Artery Vain", "Stages of heart pumping", "Diastole", "Systole", "Vasodialation", "Vasoconstriction", "Sinoatrial nodes", "Atrioventricular node"]

In [41]:
THEME = "Excretion"
TOPICS = ["Osmoregulation", "Salt Glands", "Byproducts of Nitrogen breakdown", "Functioning of Kidney", "Structure of Kidney", "RAA System", "Production of Urine"]

In [53]:
THEME = "Homeostasis"
TOPICS = ["Endocrine", "Nervous", "Mechanism of Homeostasis", "Feedback System in homeostasis", "Type of heat exchang", "Rhythms", "Minimum metabolic rate", "Torpor", "Homeotherm", "Poikilotherm"]

In [65]:
THEME = "Endocrine System"
TOPICS = ["Function of Endocrine cells", "Ways of communication", "Water soluble", "lipid soluble", "Single Endocrine Pathway", "Single neuroendocrine pathway", "Important Endocrine glands", "Gonad", "Hypothalamus", "Pineal"]

In [77]:
THEME = "Nervous System"
TOPICS = ["Central nervous system", "Peripheral nervous system ", "Types of nervous system", "Glia Cells functionality", "Anatomy of the Brain", "Gray matter, Ventricles and White matter", "Cerebellum", "Cerebrum", "Brain Stem", "Diencephalon", "ways of communication", "Working of Sleep", "Brocas area and Wernickes area", "Reflexes", "memory functionality"]

In [88]:
THEME = "Electrical Signalling"
TOPICS = ["Neurons", "Potential around membarane", "Depolarozation", "Myelin Sheath", "Synamptic transmission", "Schwan cells", "Action potential", "Rest potential"]

# Cell Bilogy

In [99]:
THEME = "Cell Structure"
TOPICS = ["Plasma Membrane", "Passive cell transport", "Active Cell Transport", "Na-K pump working", "Exo and Endocytosis", "Nucleus", "Ribosome", "Endoplasmic reticulum", "Golgi apparatus", "Lysosome", "Mitochondrion", "Chloroplast", "Cytoskeleton", "Flagella, cilia", "Tight junctions, desmosomes, gap junctions "]

In [111]:
THEME = "Cellular Signalling"
TOPICS = ["Direct Communication", "Local Communication", "Three stages of cellular signalling", "membrane receptors", "GPCR G-protien", "RTK Receptor Tyrosine Kinases", "Ion channel receptors", "Intracellular receptors", "Phosphorylation cascade (initiated by cAMP)",
          "Cyclic AMP", "Calcium ions"]

# Immunology

In [123]:
THEME = "Innate Immunology"
TOPICS = ["Function of immune system", "Defense Barriers", "Immune Cells", "Basophil and Neutrophil", "Macrophages and dendritic cells ", "Phagocytosis", "Granulocytes"]

In [5]:
THEME = "Adaptive Immunology"
TOPICS = ["Humoral immune response", "Cell-mediated immune response", "four characteristics of adaptive immune system", "T Cell functionlity", "B Cell Functionality", "Immunization", "Active and passive immunity"]

# Stress

In [16]:
THEME = "Stree Physilogy"
TOPICS = ["Stress functioning", "Factors for stress", "Allostasis", "SAM system", "HPA System", "Effect of glucose", "pancreas during stress", "Effects on Adrenalin", "cortisol", "Reactive Response", "Proactive response", "Idicators for stress", "Chronic stress", "Quantitative Measure"]

# Code

In [17]:

TEMPLATE0="""
With respect to the {theme} system, generate short descriptions not exceeding 40 words on each of the following topics {topics}.
"""

prompt_template = PromptTemplate(
    input_variables = ["theme", "topics"],
    template = TEMPLATE0
)

In [18]:
firstChain = LLMChain(llm = llm, prompt = prompt_template, output_key = "text", verbose= True)

In [19]:
TEMPLATE1="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

# tonme: Difficulty level of the question : simple| intermediate| Hard

In [20]:
prompt_template = PromptTemplate(
    input_variables = ["text","number","subject","tone","response_json"],
    template = TEMPLATE1
)

In [21]:
secondchain = LLMChain(llm = llm, prompt = prompt_template, output_key = "quiz", verbose= True)

In [71]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [10]:
#quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [11]:
#evalChain = LLMChain(llm=llm, prompt= quiz_evaluation_prompt, output_key= "review", verbose= True)

In [ ]:
#seqChain = SequentialChain(chains = [firstChain, secondchain], input_variables=["text","number","subject","tone","response_json"],output_variables=["quiz", "response"], verbose = True)

In [22]:
seqChain = SequentialChain(chains = [firstChain, secondchain], input_variables=["theme","topics","number","subject","tone","response_json"],
                           output_variables=["text", "quiz"], verbose = True)

In [106]:
#filePath = r"E:\GPT BootCamp\OPENAI\MCQGenerator\Excretion.txt"

#with open(filePath, 'r') as file:
    TEXT = file.read()

In [ ]:
#print(TEXT)

In [23]:
json.dumps(RESPONSE_JSON)

NUMBER=10
SUBJECT="Biology"
TONE="hard"

In [ ]:
#with get_openai_callback() as cb:
    response=seqChain(
        {
            "theme":TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

In [24]:
with get_openai_callback() as cb:
    response=seqChain(
        {
            "theme":THEME,
            "topics": TOPICS,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

With respect to the Stree Physilogy system, generate short descriptions not exceeding 40 words on each of the following topics ['Stress functioning', 'Factors for stress', 'Allostasis', 'SAM system', 'HPA System', 'Effect of glucose', 'pancreas during stress', 'Effects on Adrenalin', 'cortisol', 'Reactive Response', 'Proactive response', 'Idicators for stress', 'Chronic stress', 'Quantitative Measure'].


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Text:- Stress functioning: The body's response to perceived threats, activating physiological changes to help cope with the situation.
- Factors for stress: Environmental, psychological, and physical factors that can trigger the stress response.
- Allostasis: The body's ability to maintain stability through change, adjusting to stressors to maintain balance.
- SAM system: The sympathetic-adrenal-medullary 

In [61]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2493
Prompt Tokens:1736
Completion Tokens:757
Total Cost:0.004118


In [133]:
response

{'theme': 'Innate Immunology',
 'topics': ['Function of immune system',
  'Defense Barriers',
  'Immune Cells',
  'Basophil and Neutrophil',
  'Macrophages and dendritic cells ',
  'Phagocytosis',
  'Granulocytes'],
 'number': 10,
 'subject': 'Biology',
 'tone': 'hard',
 'response_json': '{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}',
 'text': '- Function of immune system: The immune system protects the body from harmful pathogens by recognizing and destroying them, as well as remembering previous encounters to mount a quicker response in the fut

In [25]:
quiz=response.get("quiz")
json.loads(quiz)

{'1': {'mcq': 'What system releases adrenaline and noradrenaline in response to stress?',
  'options': {'a': 'SAM system',
   'b': 'HPA system',
   'c': 'Pancreas',
   'd': 'Adrenal glands'},
  'correct': 'a'},
 '2': {'mcq': "What is the body's response to perceived threats, activating physiological changes to help cope with the situation?",
  'options': {'a': 'Allostasis',
   'b': 'Reactive Response',
   'c': 'Proactive Response',
   'd': 'Stress functioning'},
  'correct': 'd'},
 '3': {'mcq': 'Which hormone is released by the adrenal glands to regulate metabolism and immune function in response to stress?',
  'options': {'a': 'Insulin',
   'b': 'Cortisol',
   'c': 'Adrenaline',
   'd': 'Noradrenaline'},
  'correct': 'b'},
 '4': {'mcq': 'What type of response is an anticipatory response to stress, preparing the body for potential threats before they occur?',
  'options': {'a': 'Reactive Response',
   'b': 'Proactive Response',
   'c': 'Allostasis',
   'd': 'SAM system'},
  'correct': 

In [63]:
quiz

'\n{\n"1": {\n"mcq": "Which system regulates homeostasis through the release of hormones?",\n"options": {\n"a": "Nervous system",\n"b": "Endocrine system",\n"c": "Muscular system",\n"d": "Skeletal system"\n},\n"correct": "b"\n},\n"2": {\n"mcq": "What is the main function of the nervous system in maintaining homeostasis?",\n"options": {\n"a": "Regulating heart rate",\n"b": "Releasing hormones",\n"c": "Controlling metabolism",\n"d": "Regulating body temperature"\n},\n"correct": "a"\n},\n"3": {\n"mcq": "How is homeostasis maintained in the body?",\n"options": {\n"a": "Through the release of enzymes",\n"b": "By continuously monitoring and adjusting internal conditions",\n"c": "By increasing external temperature",\n"d": "By decreasing oxygen intake"\n},\n"correct": "b"\n},\n"4": {\n"mcq": "What is involved in the feedback system of homeostasis?",\n"options": {\n"a": "Sensors, control center, and effectors",\n"b": "Brain, lungs, and kidneys",\n"c": "Heart, liver, and pancreas",\n"d": "Muscle